# Hello "GeoPandas" World!

Let us start by importing required modules

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
#Open shapefile
plot_locations = gpd.read_file('data/sites.shp')
type(plot_locations)

Like PANDAS (DataFrame), the plot_locations variable is a **"GeoDataFrame"** and comes with several in-built methods / functions. 

Let us print the first (head) and last (tail) 6 points data.

In [ ]:
#change head to tail and see the difference
print(plot_locations.head(6))

In [ ]:
print(plot_locations.shape)

In [ ]:
#to see the bounds
print(plot_locations.total_bounds)

In [ ]:
#to view CRS of object
print(plot_locations.crs)

In [ ]:
#to view the geometry type
print(plot_locations.geom_type)

**Plot a Shapefile**

Next, you can visualize the data in your Python geodata.frame object using the .plot() method. 

Notice that you can create a plot using the geopandas base plotting using the syntax:

```python
dataframe_name.plot()
```

You can call .plot() without setting up a figure and axis object like this:

In [ ]:
plot_locations.plot()

In general it is good practice to setup an axis object so you can plot different layers together. 

When you do that you need to provide the plot function with the axis object that you want it to plot on. 

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 10))

plot_locations.plot(ax=ax1)

plt.show()

**Categorical Symbology and more** ...

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the data and add a legend
plot_locations.plot(    column='COVER',
                        categorical=True,
                        legend=True,
                        markersize=50,
                        cmap="Oranges",
                        ax=ax)
# Add a title
ax.set_title('Plot Locations\nBear River, WY')
# plt.legend()
plt.show()

You can use the cmap argument to adjust the colors of our plot. 

Below you used a colormap that is a part of the matplotlib colormap library.

![Colormaps](https://matplotlib.org/_images/lightness_01.png)

The keyword [marker](https://matplotlib.org/3.1.1/api/markers_api.html) can be used to change the symbol.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plot_locations.plot(column='COVER',
                         categorical=True,
                         legend=True,
                         marker='*',
                         markersize=65,
                         cmap='OrRd', 
                         ax=ax)
ax.set_title('Plot Locations\nBear River, WY')
plt.show()


If you get an error running the cell below, please see this issue [(#134)](https://github.com/pyproj4/pyproj/issues/134) with pyproj.

**Workaround**: open anaconda prompt, activate your environment and then start the jupyter notebook.

In [ ]:
import geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
print(world.crs)
world = world.to_crs({'init': 'epsg:3395'})
print(world.crs)

**Classwork:** Import Point & Line Shapefiles

Using the steps above, import the data/sites and data/roads shapefiles into Python. 

Answer the following questions:

    What type of Python spatial object is created when you import each layer?
    What is the CRS and extent for each object?
    Do the files contain, points, lines or polygons?
    How many spatial objects are in each file?

In [ ]:
# Import crop boundary
roads = gpd.read_file("data/road.shp")

fig, ax = plt.subplots(figsize=(10, 10))

# First setup the plot using the crop_extent layer as the base layer
roads.plot(           color='lightgrey',
                      ax=ax)

# Add another layer using geopandas syntax .plot, and calling the ax variable as the axis argument
plot_locations.plot(     column='COVER', 
                         categorical=True,
                         marker='D',
                         legend=True,
                         markersize=50,
                         cmap='Set2', 
                         ax=ax)
# Clean up axes
ax.set_title('Plot Locations along North Bear Lake Boulevard\nBear River, WY')

# ax.set_axis_off()

## Projection

Geopandas has strong **CRS transformation** and mapping options.

The example below shows the land boundaries in 2 projection systems.

In [ ]:
from matplotlib.ticker import ScalarFormatter
import numpy as np
from shapely.geometry import Point

In [ ]:
# Import graticule & world bounding box shapefile data
worldBound = gpd.read_file("data/land.shp")
bbox = gpd.read_file("data/bbox.shp")
graticule = gpd.read_file("data/graticule.shp")

# Create numpy array of x,y point locations
add_points = np.array([[-77.08, 38.89],            #Washington DC
                       [ 77.14, 28.53],            #New Delhi
                       [-0.24, 51.53]])            #London

# Turn points into list of x,y shapely points 
city_locations = [Point(xy) for xy in add_points]

# Create geodataframe using the points
city_locations = gpd.GeoDataFrame(city_locations, 
                                  columns=['geometry'],
                                  crs=worldBound.crs)

# Reproject graticules and bounding box  to robinson
worldBound_robin = worldBound.to_crs('+proj=robin')
bbox_robinson = bbox.to_crs('+proj=robin')
graticule_robinson = graticule.to_crs('+proj=robin')
city_locations_robin = city_locations.to_crs('+proj=robin')

# # Setup plot with 2 "rows" one for each map and one column
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(13, 12))

# First plot
bbox.plot(ax=ax0,
          alpha=.1,
          color='grey')
graticule.plot(ax=ax0,
               color='lightgrey')
worldBound.plot(ax=ax0,
                color='black')
city_locations.plot(ax=ax0,
                    markersize=100,
                    color='springgreen')
ax0.set(title="World Map - Geographic (long/lat degrees)")

# Second plot
bbox_robinson.plot(ax=ax1,
                   alpha=.1,
                   color='grey')
graticule_robinson.plot(ax=ax1,
                        color='lightgrey')
worldBound_robin.plot(ax=ax1,
                      color='black')
city_locations_robin.plot(ax=ax1,
                          markersize=100,
                          color='springgreen')
ax1.set(title="World Map Projected - Robinson (Meters)")

for axis in [ax1.xaxis, ax1.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

## Dissolve Polygons with Geopandas

Let us try to dissolve all the polygons in the "ganga-states.shp" file.

In [ ]:
# Import states shapefile data
states = gpd.read_file("data/ganga-states.shp")
print(states)
states.plot(cmap="Greys")

In [ ]:
# There is a column called LSAD containing zeroes. Let us merge all the features based on this column.
# Select the columns that you wish to use for the dissolve and keep.
states1 = states[['LSAD', 'geometry']]
basin  = states1.dissolve(by='LSAD')

# View the result
print(basin)
basin.reset_index().plot() 

In [ ]:
#to save the output to a shapefile is just 1 line of code!
basin.to_file("data/basin.shp")

## Spatial Analysis

Let us perform an intersection query.

Question: To find all the features in "populated_places.shp" which lie inside the newly created "basin.shp".

In [ ]:
places = gpd.read_file('data/populated_places.shp')
print('There are totally', places.shape[0], ' features in the file')

In [ ]:
# Plot the data
fig, ax = plt.subplots(figsize=(12, 8))
places.plot(markersize=0.5, 
            ax=ax)
basin.plot(alpha=0.7,
           ax=ax)

![clip](https://www.earthdatascience.org/images/courses/earth-analytics/spatial-data/vector-clip.png)

In [ ]:
#Let us define a function to clip points with polygon 
#It involves 2 steps
# - Create a mask where every point that overlaps the polygon that you wish to clip to is set to true
# - Apply that mask to filter the geopandas dataframe.
def clip(point, polygon):
    poly = polygon.geometry.unary_union
    points_clip = point[point.geometry.intersects(poly)]
    return points_clip

In [ ]:
# Clip the data using the clip_data module
clipped_points = clip(places, basin)
clipped_points.head()

In [ ]:
print('There are ', clipped_points.shape[0], ' number of points inside ganga basin')

## More complex analysis

From the highway line data (highway.shp), let us find the cumulative length in each state (ganga-states.shp).

Further let us find state-wise, road density (i.e., length of road in km per unit area in sq. km) 

In [ ]:
highway = gpd.read_file('data/highway.shp')
states = gpd.read_file('data/ganga-states.shp')

# Plot the data
fig, ax = plt.subplots(figsize=(12, 8))
states.plot(alpha=1,
            facecolor="none",
            ax=ax)
highway.plot(ax=ax)

The sjoin function performs spatial join in geopandas.

The "how" keyword has 3 possible values (left, right, inner) [[Details]](https://github.com/geopandas/geopandas/blob/master/geopandas/tools/sjoin.py#L18).

The "op" keyword has 10 possible values (\_\_eq\_\_, equals, almost_equals, contains, crosses, disjoint, intersects, overlaps, touches, within) [[Details]](https://shapely.readthedocs.io/en/stable/manual.html?highlight=binary%20predicates#binary-predicates).

In [ ]:
# Roads within region
highway_ganga = gpd.sjoin(highway, 
                          states,
                          how='inner',
                          op='intersects')

# Notice once you have joins the data - you have attributes 
# from the regions_object (i.e. NAME_1) attached to each road feature
highway_ganga[["FID", "index_right", "NAME_1"]].head()

In [ ]:
# Plot the results
fig, ax = plt.subplots(figsize=(12, 8))
states.plot(alpha=1,
            facecolor="none",
            ax=ax)
highway_ganga.plot(column='NAME_1',
                   ax=ax,
                   legend=True)

To do distance and area measurements, it is recommended that the data should be reprojected to projected coordinate system. But, for a large geographical area like India, which projection system is ideal? It spreads across 6 UTM zones (42-47). Thankfully, there is a standardized LCC based projection system [recommended by NRSC/ISRO](https://swat.tamu.edu/docs/swat/india-dataset/2012/Data_sources.txt). The EPSG code for it is [7755](http://epsg.io/7755).

In [ ]:
highway_ganga_lcc = highway_ganga.to_crs({'init': 'epsg:7755'}) 
states_lcc = states.to_crs({'init': 'epsg:7755'}) 

In [ ]:
# Turn off scientific notation
pd.options.display.float_format = '{:.4f}'.format

# Calculate the total length of road 
highway_ganga_lcc['rdlength'] = highway_ganga_lcc.length

# Sum existing columns
sub = highway_ganga_lcc[['rdlength', 'NAME_1']].groupby('NAME_1').sum()
sub

In [ ]:
states_lcc['area'] = states_lcc.area/1e6
states_lcc

In [ ]:
sub = sub.merge(states_lcc, on='NAME_1')
sub[['NAME_1', 'rdlength', 'area']]

In [ ]:
sub['highway_length_by_area'] = sub['rdlength']/sub['area']
sub[['NAME_1', 'highway_length_by_area']].sort_values(by=['highway_length_by_area'])

**Homework**: Try out [handling missing spatial attribute data](https://www.earthdatascience.org/courses/earth-analytics-python/spatial-data-vector-shapefiles/missing-data-vector-data-in-python/)